In [196]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [197]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [198]:
rdd = spark.sparkContext.textFile('googleplaystore.csv')
rdd.take(5)

['App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver',
 'Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up',
 'Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up',
 '"U Launcher Lite – FREE Live Cool Themes, Hide Apps",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up',
 'Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up']

In [199]:
sqlContext = SQLContext(sc)
dataframe = sqlContext.read.csv('googleplaystore.csv', header=True)
apps = dataframe.rdd
apps.take(5)

/home/alejo/.local/lib/python3.10/site-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


[Row(App='Photo Editor & Candy Camera & Grid & ScrapBook', Category='ART_AND_DESIGN', Rating='4.1', Reviews='159', Size='19M', Installs='10,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design', Last Updated='January 7, 2018', Current Ver='1.0.0', Android Ver='4.0.3 and up'),
 Row(App='Coloring book moana', Category='ART_AND_DESIGN', Rating='3.9', Reviews='967', Size='14M', Installs='500,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design;Pretend Play', Last Updated='January 15, 2018', Current Ver='2.0.0', Android Ver='4.0.3 and up'),
 Row(App='U Launcher Lite – FREE Live Cool Themes, Hide Apps', Category='ART_AND_DESIGN', Rating='4.7', Reviews='87510', Size='8.7M', Installs='5,000,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design', Last Updated='August 1, 2018', Current Ver='1.2.4', Android Ver='4.0.3 and up'),
 Row(App='Sketch - Draw & Paint', Category='ART_AND_DESIGN', Rating='4.5', Reviews='215644', S

In [200]:
apps.filter(lambda x: x.App == 'Photo Editor & Candy Camera & Grid & ScrapBook').collect()

[Row(App='Photo Editor & Candy Camera & Grid & ScrapBook', Category='ART_AND_DESIGN', Rating='4.1', Reviews='159', Size='19M', Installs='10,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design', Last Updated='January 7, 2018', Current Ver='1.0.0', Android Ver='4.0.3 and up')]

### 1. ¿Cuál es la categoría con mayor cantidad de reviews promedio en sus aplicaciones? ¿Por qué? ⭐

### 2. ¿Cuál es el género más común de las aplicaciones? Indique la cantidad de aplicaciones con dicho género ⭐

In [201]:
apps = apps.map(lambda x: (x.App, list(x.Genres.split(';'))))
apps = apps.flatMap(lambda x: [(x[0], item) for item in x[1]])

In [202]:
apps.map(lambda x: (x[1], 1)).reduceByKey(lambda x,y: x+y).takeOrdered(1)

[('Action', 382)]

### 7. De las apps que tienen el nombre “FREE” ¿Cuál es la menos puntuada? (Rating) Si hay más de una, mostrar cualquiera ⭐
(cambiamos el enunciado a 'las apps que tienen EN el nombre "FREE"...)

In [203]:
rdd = spark.sparkContext.textFile('googleplaystore.csv')
rdd.take(5)
sqlContext = SQLContext(sc)
dataframe = sqlContext.read.csv('googleplaystore.csv', header=True)
apps = dataframe.rdd
apps.take(5)

[Row(App='Photo Editor & Candy Camera & Grid & ScrapBook', Category='ART_AND_DESIGN', Rating='4.1', Reviews='159', Size='19M', Installs='10,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design', Last Updated='January 7, 2018', Current Ver='1.0.0', Android Ver='4.0.3 and up'),
 Row(App='Coloring book moana', Category='ART_AND_DESIGN', Rating='3.9', Reviews='967', Size='14M', Installs='500,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design;Pretend Play', Last Updated='January 15, 2018', Current Ver='2.0.0', Android Ver='4.0.3 and up'),
 Row(App='U Launcher Lite – FREE Live Cool Themes, Hide Apps', Category='ART_AND_DESIGN', Rating='4.7', Reviews='87510', Size='8.7M', Installs='5,000,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design', Last Updated='August 1, 2018', Current Ver='1.2.4', Android Ver='4.0.3 and up'),
 Row(App='Sketch - Draw & Paint', Category='ART_AND_DESIGN', Rating='4.5', Reviews='215644', S

In [204]:
apps.filter(lambda x: 'FREE' in x.App).map(lambda x: (x.App, x.Rating)).takeOrdered(5, lambda x: float(x[1]))

[('Remote For ATT U-verse TV - NOW FREE', '2.5'),
 ('CB Frequencies FREE!', '3.1'),
 ('AC Remote for Daikin - NOW FREE', '3.5'),
 ('AppClose - The #1 FREE Co-Parenting App', '3.5'),
 ('AC Remote for O General - NOW FREE', '3.7')]

### 8. Devolver la aplicación más polarizante. tener en cuenta el promedio de polarización de las reviews de cada aplicación ⭐

In [325]:
rdd = spark.sparkContext.textFile('googleplaystore_user_reviews.csv')
rdd.take(5)
sqlContext = SQLContext(sc)
dataframe = sqlContext.read.csv('googleplaystore_user_reviews.csv', header=True)
reviews = dataframe.rdd
reviews.take(5)

/home/alejo/.local/lib/python3.10/site-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


[Row(App='10 Best Foods for You', Translated_Review='"I like eat delicious food. That\'s I\'m cooking food myself, case ""10 Best Foods"" helps lot', Sentiment=' also ""Best Before (Shelf Life)"""', Sentiment_Polarity='Positive', Sentiment_Subjectivity='1.0'),
 Row(App='10 Best Foods for You', Translated_Review='This help eating healthy exercise regular basis', Sentiment='Positive', Sentiment_Polarity='0.25', Sentiment_Subjectivity='0.28846153846153844'),
 Row(App='10 Best Foods for You', Translated_Review='nan', Sentiment='nan', Sentiment_Polarity='nan', Sentiment_Subjectivity='nan'),
 Row(App='10 Best Foods for You', Translated_Review='Works great especially going grocery store', Sentiment='Positive', Sentiment_Polarity='0.4', Sentiment_Subjectivity='0.875'),
 Row(App='10 Best Foods for You', Translated_Review='Best idea us', Sentiment='Positive', Sentiment_Polarity='1.0', Sentiment_Subjectivity='0.3')]

In [326]:
import math

def func(x):
    try:
        x = float(x)
        return True
    except:
        return False

rdd = reviews.filter(lambda x: func(x.Sentiment_Polarity))
rdd = rdd.map(lambda x: (x.App, float(x.Sentiment_Polarity), 1))
rdd = rdd.filter(lambda x: not math.isnan(x[1]))

In [328]:
rdd.take(5)

[('10 Best Foods for You', 0.25, 1),
 ('10 Best Foods for You', 0.4, 1),
 ('10 Best Foods for You', 1.0, 1),
 ('10 Best Foods for You', 1.0, 1),
 ('10 Best Foods for You', 0.6000000000000001, 1)]

In [329]:
rdd = rdd.map(lambda x: (x[0], (x[1], x[2])))
rdd.take(10)

[('10 Best Foods for You', (0.25, 1)),
 ('10 Best Foods for You', (0.4, 1)),
 ('10 Best Foods for You', (1.0, 1)),
 ('10 Best Foods for You', (1.0, 1)),
 ('10 Best Foods for You', (0.6000000000000001, 1)),
 ('10 Best Foods for You', (0.0, 1)),
 ('10 Best Foods for You', (0.0, 1)),
 ('10 Best Foods for You', (0.7, 1)),
 ('10 Best Foods for You', (0.2, 1)),
 ('10 Best Foods for You', (0.75, 1))]

In [337]:
rdd.reduceByKey(lambda x,y: (x[0] + y[0], x[1] + y[1])).map(lambda x: (x[0], x[1][0] / x[1][1])).takeOrdered(5, lambda x: - x[1])

[('HomeWork', 1.0),
 ('Google Slides', 0.9333333333333332),
 ('Daily Workouts - Exercise Fitness Routine Trainer', 0.8),
 ('Bed Time Fan - White Noise Sleep Sounds', 0.78125),
 ('Cameringo Lite. Filters Camera', 0.7702690972222221)]